In [1]:
import locale
locale.getpreferredencoding()

'UTF-8'

In [2]:
# import glob
import os
import pathlib
import pandas as pd
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from IPython.display import Image, display
import cv2
import magic
%matplotlib inline

In [19]:

data_path = "../data"
city = "gz"
data_city_path = data_path+ "/"+city

data_p = pathlib.Path(data_city_path)

In [20]:
images_list = [str(p) for p in list(data_p.glob("./*"))]

In [21]:
print(images_list[0])

../data/gz/其他垃圾_水果硬壳_椰子壳_37.jpg


In [22]:
images_list = [str(p) for p in list(data_p.glob("./*"))]
filenames_list = [p.name for p in list(data_p.glob("./*"))]
suffixes_list = [p.suffix for p in list(data_p.glob("./*"))]

In [23]:
# Check length
print(len(images_list) == len(filenames_list) == len(suffixes_list))
print(len(images_list))

True
8250


In [24]:
# Parse classes from filename

r_list = []
for i, a in enumerate(filenames_list):
    # print(a)
    try:
        ext = suffixes_list[i]
        basename = a.split(".")[0]
        # print(basename)
        name_seg = basename.split("_")
        sub_id = name_seg.pop()
        sample = name_seg.pop()
        labels = {"level"+str(i):l for i,l in enumerate(name_seg)}
        r = {"_filename": a, "_format": ext, "sample": sample, "spider_id": sub_id, "city": city}
        res = {**r, **labels}
        # print(res)
        r_list.append(res)
    except:
        print(i)

In [25]:
r_df = pd.DataFrame(r_list)

In [26]:
r_df.head()

,_filename,_format,sample,spider_id,city,level0,level1
0,其他垃圾_水果硬壳_椰子壳_37.jpg,.jpg,椰子壳,37,gz,其他垃圾,水果硬壳
1,其他垃圾_混杂_土培植物_26.jpg,.jpg,土培植物,26,gz,其他垃圾,混杂
2,厨余垃圾_糖果糕点_糖果_25.jpg,.jpg,糖果,25,gz,厨余垃圾,糖果糕点
3,其他垃圾_坚果_核桃壳_28.jpg,.jpg,核桃壳,28,gz,其他垃圾,坚果
4,其他垃圾_废旧衣服及其他纺织品_内衣裤_46.jpg,.jpg,内衣裤,46,gz,其他垃圾,废旧衣服及其他纺织品


In [27]:
def is_readable_image_data(data_file):
    import magic
    try:
        magic_msg = magic.from_file(str(data_file))
    except Exception as e:
        print(e)
        return False
    if (
        magic_msg.startswith("JPEG image data") 
        or magic_msg.startswith("PNG image data") 
        # or magic_msg.startswith("GIF image data")
       ):
        return True
    else:
        return False

In [33]:
r_df['is_readable_image'] = r_df.apply(lambda x: is_readable_image_data(str(data_path+"/"+x['city']+"/"+x['_filename'])), axis=1)

b'JPEG image data, JFIF standard 1.01, resolution (DPI), density 96x96, segment length 16, Exif Standard: [TIFF image data, little-endian, direntries=15, manufacturer=Canon, model=Canon EOS 5D Mark III, orientation=upper-left, xresolution=8, yresolution=16, resolutionunit=2, datetime=2016:04:05 19:09:18, GPS-Data] name use count (30) exceeded'


In [37]:
print(r_df[~r_df['is_readable_image']].shape)
r_df[~r_df['is_readable_image']].head()

(148, 8)


,_filename,_format,sample,spider_id,city,level0,level1,is_readable_image
30,其他垃圾_混杂_废弃化妆品_27.jpg,.jpg,废弃化妆品,27,gz,其他垃圾,混杂,False
135,可回收物_废织物_手提包_29.jpg,.jpg,手提包,29,gz,可回收物,废织物,False
140,可回收物_废金属_金属元件_1.jpg,.jpg,金属元件,1,gz,可回收物,废金属,False
220,可回收物_废织物_床单_24.jpg,.jpg,床单,24,gz,可回收物,废织物,False
249,可回收物_废玻璃_玻璃盘_41.jpg,.jpg,玻璃盘,41,gz,可回收物,废玻璃,False


In [38]:
print(r_df.shape)
print(r_df['is_readable_image'].value_counts())

(8250, 8)
True     8102
False     148
Name: is_readable_image, dtype: int64


In [41]:
(r_df['city'] + "/" +r_df['_filename']).tolist()

['gz/其他垃圾_水果硬壳_椰子壳_37.jpg',
 'gz/其他垃圾_混杂_土培植物_26.jpg',
 'gz/厨余垃圾_糖果糕点_糖果_25.jpg',
 'gz/其他垃圾_坚果_核桃壳_28.jpg',
 'gz/其他垃圾_废旧衣服及其他纺织品_内衣裤_46.jpg',
 'gz/有害垃圾_废灯管类_荧光灯管_3.jpg',
 'gz/厨余垃圾_水培植物_水培植物_15.jpg',
 'gz/可回收物_废织物_衣服_26.jpg',
 'gz/厨余垃圾_肉蛋食品_鱼_51.jpg',
 'gz/可回收物_废纸类_信封_17.jpg',
 'gz/可回收物_废金属_螺丝刀_2.jpg',
 'gz/其他垃圾_骨头贝壳_螺蛳壳_27.jpeg',
 'gz/可回收物_废纸类_包装纸_21.jpg',
 'gz/可回收物_废纸类_挂历_32.jpg',
 'gz/可回收物_废纸类_包装纸_41.jpg',
 'gz/可回收物_废金属_金属衣架_10.png',
 'gz/可回收物_废塑料_奶瓶_33.jpg',
 'gz/人像_selfie_922.jpg',
 'gz/人像_selfie_903.jpg',
 'gz/可回收物_废织物_衣服_10.jpg',
 'gz/可回收物_废纸类_包装纸_18.jpg',
 'gz/厨余垃圾_肉蛋食品_蟹_57.jpg',
 'gz/其他垃圾_混杂_干燥剂_15.jpg',
 'gz/人像_selfie_814.jpg',
 'gz/其他垃圾_水果硬壳_椰子壳_16.png',
 'gz/其他垃圾_塑料_垃圾袋_30.jpg',
 'gz/可回收物_废织物_床单_33.jpg',
 'gz/人像_selfie_301.jpg',
 'gz/厨余垃圾_宠物饲料_55.jpg',
 'gz/可回收物_废玻璃_玻璃盘_18.jpg',
 'gz/其他垃圾_混杂_废弃化妆品_27.jpg',
 'gz/其他垃圾_混杂_烟头_45.jpg',
 'gz/可回收物_废塑料_泡沫塑料_39.jpg',
 'gz/其他垃圾_坚果_花生壳_47.png',
 'gz/可回收物_废金属_废旧电线_24.jpg',
 'gz/其他垃圾_水果硬壳_椰子壳_56.jpg',
 'gz/厨余垃圾_肉蛋食品_虾_14.jpg',
 'gz/可回收物

In [42]:
filenames = [data_path + "/" +fname for fname in (r_df['city'] + "/" +r_df['_filename']).tolist()]

def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    return image

def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    return preprocess_image(image)

def is_completed_image(path):
    try:
        load_and_preprocess_image(path)
        return True
    except Exception as e:
        return False

def get_corrupted_image_files(file_list):
    cr_list = []
    
    for i, f in enumerate(filenames):
        try:
            load_and_preprocess_image(f)
        except Exception as e:
            cr_list.append((i, f))
            
    return cr_list

# corrupted_images = get_corrupted_image_files(filenames)
r_df['is_completed_image'] = r_df.apply(lambda x: is_completed_image(str(data_path+"/"+x['city']+"/"+x['_filename'])), axis=1)

In [44]:
r_df.to_csv("../_"+city+"_filename_dict.csv", index=False)

In [45]:
r_df['level0'].value_counts()

可回收物    2403
其他垃圾    2185
人像      1500
有害垃圾    1141
厨余垃圾    1021
Name: level0, dtype: int64

In [46]:
a = {}
if a:
    print(1)
else:
    print(2)

2
